In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import concurrent.futures
import glob
import io
import seaborn as sns
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
from os import listdir

In [2]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'

In [3]:
files = pd.read_csv(path + '/all_files_cleaned.csv')
files = files.drop(columns={'Unnamed: 0'})
files

,schema_type,column_name,data_type,percentage,filename,num_tables,prob_matches
0,name/ Product_stats.csv,name,string,0.994361,Product_stats.csv,432728.0,product_name
1,offers/ Product_stats.csv,offers,string,0.964249,Product_stats.csv,419624.0,offer
2,brand/ Product_stats.csv,brand,string,0.403728,Product_stats.csv,175695.0,product_name
3,headline/ CreativeWork_stats.csv,headline,string,0.939871,CreativeWork_stats.csv,76451.0,headline
4,author/ CreativeWork_stats.csv,author,string,0.776499,CreativeWork_stats.csv,63162.0,person_name
...,...,...,...,...,...,...,...
202,numtracks/ MusicAlbum_stats.csv,numtracks,int,0.415808,MusicAlbum_stats.csv,121.0,number
203,menu/ Restaurant_stats.csv,menu,string,0.104581,Restaurant_stats.csv,121.0,text_description
204,contactpoint/ Person_stats.csv,contactpoint,string,0.009353,Person_stats.csv,120.0,text_description
205,email/ Hotel_stats.csv,email,string,0.107664,Hotel_stats.csv,118.0,email


In [4]:
columns_sorted = files.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
columns_sorted.filename = columns_sorted.filename.str.split('_', expand = True) [0]
columns_sorted.index = columns_sorted['filename']
columns_sorted

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [13]:
def count_move_small_tables(table):
    df = pd.read_json(os.path.join(input+table), compression='gzip', lines=True)
    if len(df)<=10:
        df.to_json(os.path.join(output+'/under_10_rows/'+table), compression = 'gzip', lines = True)
        print(len(df))
        return
    else:
        row_list.extend([table,len(df)])
        print(row_list)
   
       

In [21]:
row_list = list()
for file in files:
    #print(index)
    input = '../../src/data/data/'+index+'/All/'
    output = '../../src/data/data/'+index
    if not os.path.exists(output):
        os.makedirs(output)
    #for _ in tqdm(count_move_small_tables, [file for file in os.columns_sorted(input) if file.endswith('.json.gz')]), total=len([file for file in os.columns_sortedinput) if file.endswith('.json.gz')]):
     #   pass
    df = pd.read_json(os.path.join(input, '{}'.format(file)), compression='gzip', lines=True)
    if len(df) <= 10:
        df.to_json(os.path.join(output+'/under_10_rows/'+file), compression = 'gzip', lines = True)
        print(len(df))
    elif len(df)>10:
        row_list.extend([table,len(df)])
        print(len([file for file in os.listdir(output) if file.endswith('.json.gz')]))

ValueError: Expected object or value

In [ ]:
#count rows in dataframe
path = '../../src/data/data/Books/All/'

for file in files:
        df = pd.read_json(os.path.join(path, '{}'.format(file)), compression='gzip', lines=True)
        if len(df) <= 10:
           df.to_json(os.path.join(path+'/under_10_rows/'+file), compression = 'gzip', lines = True)
        print(len(df))
        else:
          
          #os.remove(os.path.join(output, '{}'.format(file)))

files = [file for file in os.listdir(path) if file.endswith('.json.gz')]
print(file, len(files))

In [5]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/schemafiltereddata/Book/'

In [6]:
files = [file for file in os.listdir(path) if file.endswith('.json.gz')]

In [8]:
row_df = pd.DataFrame
for file in files:
        df = pd.read_json(os.path.join(path, '{}'.format(file)), compression='gzip', lines=True)
        if len(df) <= 10:
           df.to_json(os.path.join(path+'/under_10_rows/'+file), compression = 'gzip', lines = True)
        print(len(df))

        row_list.extend([file,len(df)])
        #print(row_list)
   
          
          #os.remove(os.path.join(output, '{}'.format(file)))

#files = [file for file in os.listdir(path) if file.endswith('.json.gz')]
print(file, len(files))
print(row_list)

274
398
248
12194
26
202
6488
262
410
150
1130
135
178
1045
2748
608
4355
544
491
764
247
13910
2015
351
1533
1033
224
171
26
2616
63
1293
526
1859
189
37
12232
238
1321
1060
41
466
536
111
1749
1430
948
36
222
50
2878
3276
44
153
1323
1433
309
1423
178
263
52
71
800
28
5025
1377
83
1195
968
362
3621
1814
682
2249
3750
207
283
47
103
304
457
202
3127
1597
433
1156
127
619
496
223
243
793
372
1184
3713
8442
2063
1813
1420
15922
556
5170
952
325
157
3352
89
148
28
5839
182
2104
4717
4963
1150
4896
245
216
106
1624
738
1880
964
657
30
235
1085
219
132
15898
181
1048
38
127
636
25
18330
1135
1346
2783
175
425
1336
411
404
347
135
476
5797
102
1317
3388
327
82
4193
1322
1083
3029
4322
1745
1107
355
5890
945
718
2786
172
1026
905
148
137
1246
114
136
143
124
141
1208
2029
52
1019
697
1925
412
42
1962
2129
2984
1357
209
1199
1803
1324
8314
644
257
560
707
33663
1167
4311
951
4501
496
749
530
185
3877
2449
147
50
1018
1334
978
1095
993
35
312
32
102
941
531
1763
404
283
34
295
998
74
6071
64
3

In [11]:
books = pd.DataFrame (row_list, columns = ['length'])
books.head()

,length
0,Book_foldvaribooks.com_September2020.json.gz
1,274
2,Book_virago.co.uk_September2020.json.gz
3,398
4,Book_borgantiquarian.com_September2020.json.gz


In [16]:
row_df = pd.DataFrame ()
for file in files:
        df = pd.read_json(os.path.join(path, '{}'.format(file)), compression='gzip', lines=True)
        if len(df) <= 10:
           df.to_json(os.path.join(path+'/under_10_rows/'+file), compression = 'gzip', lines = True)
        print(len(df))

        row_df = row_df.append({'file': file, 'length': len(df)}, ignore_index=True)
        #print(row_list)
   
          
          #os.remove(os.path.join(output, '{}'.format(file)))

#files = [file for file in os.listdir(path) if file.endswith('.json.gz')]
print(file, len(files))
print(row_df)

274
398
248
12194
26
202
6488
262
410
150
1130
135
178
1045
2748
608
4355
544
491
764
247
13910
2015
351
1533
1033
224
171
26
2616
63
1293
526
1859
189
37
12232
238
1321
1060
41
466
536
111
1749
1430
948
36
222
50
2878
3276
44
153
1323
1433
309
1423
178
263
52
71
800
28
5025
1377
83
1195
968
362
3621
1814
682
2249
3750
207
283
47
103
304
457
202
3127
1597
433
1156
127
619
496
223
243
793
372
1184
3713
8442
2063
1813
1420
15922
556
5170
952
325
157
3352
89
148
28
5839
182
2104
4717
4963
1150
4896
245
216
106
1624
738
1880
964
657
30
235
1085
219
132
15898
181
1048
38
127
636
25
18330
1135
1346
2783
175
425
1336
411
404
347
135
476
5797
102
1317
3388
327
82
4193
1322
1083
3029
4322
1745
1107
355
5890
945
718
2786
172
1026
905
148
137
1246
114
136
143
124
141
1208
2029
52
1019
697
1925
412
42
1962
2129
2984
1357
209
1199
1803
1324
8314
644
257
560
707
33663
1167
4311
951
4501
496
749
530
185
3877
2449
147
50
1018
1334
978
1095
993
35
312
32
102
941
531
1763
404
283
34
295
998
74
6071
64
3

In [19]:
row_df

,file,length
0,Book_foldvaribooks.com_September2020.json.gz,274.0
1,Book_virago.co.uk_September2020.json.gz,398.0
2,Book_borgantiquarian.com_September2020.json.gz,248.0
3,Book_downpour.com_September2020.json.gz,12194.0
4,Book_marasworld.com_September2020.json.gz,26.0
...,...,...
973,Book_benjamins.com_September2020.json.gz,1795.0
974,Book_nollegiu.com_September2020.json.gz,3175.0
975,Book_nosegraze.com_September2020.json.gz,153.0
976,Book_hachettebooks.com_September2020.json.gz,583.0


In [22]:
row_df.mean(axis = 1).mean()

1512.4744376278118